In [ ]:
import jax, flax, numpy
import flax.linen as nn
import orbax
from notebookinit import *
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import optax
import tiktoken
import lib_kata
from tqdm.auto import tqdm

2023-12-30 22:16:21.243478: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 22:16:21.243507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 22:16:21.244415: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 22:16:21.984681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
text_encoded, text_encoder, text_decoder, vocabulary_size = lib_kata.load_dataset_and_tokenize()

vocabulary: 
 ,e,t,o,a,i,h,s,r,n,
,l,d,u,m,y,w,,,c,f,g,b,p,:,k,v,.,',;,?,!,-,j,q,x,z,3,&,$ 
 size: 39 chars
first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [ ]:
context_size=128
embedd_features=12
learning_rate=1e-2
hidden_features_per_layer = 32


In [ ]:
X_all,y_all = lib_kata.make_training_Xy(text_encoded, context_size=context_size)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.05, shuffle=False, random_state=42)

In [ ]:
lib_kata.preview_Xy(X=X_train, y=y_train, text_decoder=text_decoder, count=5, offset=0)

[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
 17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
 10 15  3 13  0  4  8  1  0  4 11 11  0  8  1  7  3 11 25  1 12  0  8  4
  2  6  1  8  0  2  3  0]  ->  12  |  first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to  _ d
[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0
 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4
  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24 17
  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10
 15  3 13  0  4  8  1  0  4 11 11  0  8  1  7  3 11 25  1 12  0  8  4  2
  6  1  8  0  2  3  0 12]  ->  5  |  irst citizen:
before we proceed any further, hear me speak.

a

In [ ]:
vocabulary_size

39

In [ ]:
embedder = nn.Embed(num_embeddings=vocabulary_size, features=embedd_features)

In [ ]:
embedder

Embed(
    # attributes
    num_embeddings = 39
    features = 12
    dtype = None
    param_dtype = float32
    embedding_init = init
)

In [ ]:
embedder_example_param = embedder.init(jax.random.PRNGKey(0), X_train[0])

In [ ]:
example_X = X_train[0].reshape([1,context_size])
example_X.shape

(1, 128)

In [ ]:
tabulate_fn = nn.tabulate(embedder, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(example_X))

                                 Embed Summary                                  
┏━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path ┃ module ┃ inputs       ┃ outputs           ┃ params                    ┃
┡━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      │ Embed  │ int32[1,128] │ float32[1,128,12] │ embedding: float32[39,12] │
│      │        │              │                   │                           │
│      │        │              │                   │ 468 (1.9 KB)              │
├──────┼────────┼──────────────┼───────────────────┼───────────────────────────┤
│      │        │              │             Total │ 468 (1.9 KB)              │
└──────┴────────┴──────────────┴───────────────────┴───────────────────────────┘
                                                                                
                         Total Parameters: 468 (1.9 KB)                         

In [ ]:
X_small,y_small = lib_kata.make_training_Xy(text_encoded[0:2000], context_size=context_size)

example_X=X_small[0].reshape([1,context_size])

example_target_y=y_small[0].reshape([-1,1])
example_target_y_logits = nn.activation.one_hot(example_target_y,vocabulary_size)
example_target_y_logits

Array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [ ]:
text_decoder(jnp.argmax(example_target_y_logits).reshape([-1,1]))

'd'

In [ ]:

    
class SimpleNN1(nn.Module):
    # batch size is implied from data
    context_size: int = 8    
    embedd_features: int = 5
    hidden_features_per_layer: int = 16
    vocabulary_size: int = 39
    

    def setup(self):
        # print(self.context_size, self.embedd_features, "->", self.context_size * self.embedd_features,)
        pass
    
    
    @nn.compact
    def __call__(self, x):
        # first, embedd the input
        y0 = nn.Embed(name='embedding',num_embeddings=self.vocabulary_size, features=self.embedd_features)(x)
        # flatten the input vector, as there is no conceptual difference between the token's identity and token's feature here:        
        h = y0.reshape([-1,self.context_size*self.embedd_features])
        
        h = nn.Dense(name='layer_1',features=self.hidden_features_per_layer)(h)
        h = nn.leaky_relu(h)        
        h = nn.Dense(name='layer_2',features=self.hidden_features_per_layer)(h)
        h = nn.leaky_relu(h)
        h = nn.Dense(name='layer_3',features=self.hidden_features_per_layer)(h)
        h = nn.leaky_relu(h)  
        
        # final dense layer down to prediction logits. Note that there might be more logits than in the hidden state, that's OK.
        h = nn.Dense(name='layer_final',features=self.vocabulary_size)(h)                        
        # h = nn.activation.softmax(h)        
        y = h
        return y


simpleNN = SimpleNN1(context_size=context_size, embedd_features=embedd_features, hidden_features_per_layer=hidden_features_per_layer)
simpleNN_params = simpleNN.init(jax.random.PRNGKey(0), example_X)
tabulate_fn = nn.tabulate(simpleNN, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(example_X))
print(f'{simpleNN.apply(simpleNN_params,example_X).shape=}')
predicted_logits = simpleNN.apply(simpleNN_params,example_X)
predicted_example_token = jnp.argmax(simpleNN.apply(simpleNN_params,example_X), axis=1)
predicted_logits_str = ", ".join([f'{float(predicted_logits[1,x]):0.2f}' for x in range(predicted_logits.shape[1])])
print(predicted_logits_str)
print(f'predicted token: ', predicted_example_token, "-> >>>", text_decoder(predicted_example_token),"<<<")
print(f'   target token: ', example_target_y, f"confidence: {float(predicted_logits[0,predicted_example_token][0]):0.3f}")


                                      SimpleNN1 Summary                                      
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path        ┃ module    ┃ inputs          ┃ outputs           ┃ params                    ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             │ SimpleNN1 │ int32[1,128]    │ float32[1,39]     │                           │
├─────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ embedding   │ Embed     │ int32[1,128]    │ float32[1,128,12] │ embedding: float32[39,12] │
│             │           │                 │                   │                           │
│             │           │                 │                   │ 468 (1.9 KB)              │
├─────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ layer_1     │ Dense     │ float32[1,1536] │ float32[1,32]     │ bias: float32[32]         │
│             │           │                 │                   │ kernel: float32[1536,32]  │
│             │           │                 │                   │                           │
│             │           │                 │                   │ 49,184 (196.7 KB)         │
├─────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ layer_2     │ Dense     │ float32[1,32]   │ float32[1,32]     │ bias: float32[32]         │
│             │           │                 │                   │ kernel: float32[32,32]    │
│             │           │                 │                   │                           │
│             │           │                 │                   │ 1,056 (4.2 KB)            │
├─────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ layer_3     │ Dense     │ float32[1,32]   │ float32[1,32]     │ bias: float32[32]         │
│             │           │                 │                   │ kernel: float32[32,32]    │
│             │           │                 │                   │                           │
│             │           │                 │                   │ 1,056 (4.2 KB)            │
├─────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ layer_final │ Dense     │ float32[1,32]   │ float32[1,39]     │ bias: float32[39]         │
│             │           │                 │                   │ kernel: float32[32,39]    │
│             │           │                 │                   │                           │
│             │           │                 │                   │ 1,287 (5.1 KB)            │
├─────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│             │           │                 │             Total │ 53,051 (212.2 KB)         │
└─────────────┴───────────┴─────────────────┴───────────────────┴───────────────────────────┘
                                                                                             
                             Total Parameters: 53,051 (212.2 KB)                             




simpleNN.apply(simpleNN_params,example_X).shape=(1, 39)
-0.07, 0.17, -0.05, 0.17, 0.07, 0.09, -0.16, -0.10, -0.12, -0.08, -0.03, -0.26, -0.27, -0.08, 0.08, 0.16, -0.20, 0.32, -0.10, 0.13, -0.04, -0.02, 0.04, 0.08, 0.06, -0.10, 0.03, 0.15, -0.26, 0.02, 0.15, -0.15, -0.17, -0.06, 0.07, -0.11, -0.11, -0.14, -0.13
predicted token:  [17] -> >>> , <<<
   target token:  [[12]] confidence: 0.319


In [ ]:
example_target_y

array([[12]], dtype=int32)

In [ ]:
# def cross_entropy_loss(logits, labels):
#     """Returns cross-entropy loss."""
#     assert jnp.all(logits.shape==labels.shape)
#     return -jnp.mean(jnp.sum(logits * labels, axis=1))

In [ ]:
# predicted_logits

In [ ]:
predicted_logits.shape

(1, 39)

In [ ]:
example_target_y.shape

(1, 1)

In [ ]:
# demo the library softmax error checker
optax.softmax_cross_entropy_with_integer_labels(predicted_logits, example_target_y[0])

Array([3.9150844], dtype=float32)

In [ ]:
# example loss between the target and predicted
jnp.log10(optax.softmax_cross_entropy_with_integer_labels(predicted_logits, example_target_y[0]))

Array([0.59274113], dtype=float32)

In [ ]:
# I think I am ready to do a training step

In [ ]:
train_minibatch_size = 8*4096
examples_per_epoch = len(X_train) - train_minibatch_size  # how many unique examples of input-output pairs can we produce out of the train-time dataset
batches_per_epoch =  len(X_train) // train_minibatch_size
print(f'batches_per_epoch per epoch: {batches_per_epoch} with minibatch size of {train_minibatch_size}')
starter_X_minibatch = jnp.array(X_train[0:train_minibatch_size])
starter_y_minibatch = jnp.array(X_train[0:train_minibatch_size])

simpleNN_minibatched = SimpleNN1(context_size=context_size, embedd_features=embedd_features, hidden_features_per_layer=hidden_features_per_layer)
simpleNN_minibatched_params = simpleNN.init(jax.random.PRNGKey(0), starter_X_minibatch)
tabulate_fn = nn.tabulate(simpleNN_minibatched, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(starter_X_minibatch))

batches_per_epoch per epoch: 32 with minibatch size of 32768


                                          SimpleNN1 Summary                                          
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path        ┃ module    ┃ inputs              ┃ outputs               ┃ params                    ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             │ SimpleNN1 │ int32[32768,128]    │ float32[32768,39]     │                           │
├─────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ embedding   │ Embed     │ int32[32768,128]    │ float32[32768,128,12] │ embedding: float32[39,12] │
│             │           │                     │                       │                           │
│             │           │                     │                       │ 468 (1.9 KB)              │
├─────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ layer_1     │ Dense     │ float32[32768,1536] │ float32[32768,32]     │ bias: float32[32]         │
│             │           │                     │                       │ kernel: float32[1536,32]  │
│             │           │                     │                       │                           │
│             │           │                     │                       │ 49,184 (196.7 KB)         │
├─────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ layer_2     │ Dense     │ float32[32768,32]   │ float32[32768,32]     │ bias: float32[32]         │
│             │           │                     │                       │ kernel: float32[32,32]    │
│             │           │                     │                       │                           │
│             │           │                     │                       │ 1,056 (4.2 KB)            │
├─────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ layer_3     │ Dense     │ float32[32768,32]   │ float32[32768,32]     │ bias: float32[32]         │
│             │           │                     │                       │ kernel: float32[32,32]    │
│             │           │                     │                       │                           │
│             │           │                     │                       │ 1,056 (4.2 KB)            │
├─────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ layer_final │ Dense     │ float32[32768,32]   │ float32[32768,39]     │ bias: float32[39]         │
│             │           │                     │                       │ kernel: float32[32,39]    │
│             │           │                     │                       │                           │
│             │           │                     │                       │ 1,287 (5.1 KB)            │
├─────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│             │           │                     │                 Total │ 53,051 (212.2 KB)         │
└─────────────┴───────────┴─────────────────────┴───────────────────────┴───────────────────────────┘
                                                                                                     
                                 Total Parameters: 53,051 (212.2 KB)                                 

In [ ]:
model_param_count = numpy.sum(numpy.array([numpy.prod(x.shape) for x in jax.tree_util.tree_leaves(simpleNN_minibatched_params)]))
model_size_str = f'{model_param_count/1e3:0.0f}k'
print(f'model size: {model_size_str}')
train_data_size = len(X_train)
print(f'model size to dataset size ratio: {model_param_count/train_data_size:0.3f}')

model size: 53k
model size to dataset size ratio: 0.050


In [ ]:
def prep_minibatch(minibatch_idx=0):
    minibatch_ptr = minibatch_idx*train_minibatch_size
    X_minibatch = jnp.array(X_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch = jnp.array(y_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch_target_logits = nn.activation.one_hot(y_minibatch,vocabulary_size)
    return X_minibatch, y_minibatch, y_minibatch_target_logits
minibatch_idx = 4
minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)

In [ ]:
# debug the X/y pair correctness
# for idx_step in range(3):
#     print(minibatch_X[idx_step,:], "->", minibatch_y[idx_step])

In [ ]:
model_starter_params = simpleNN_minibatched_params
model_moving_params = model_starter_params.copy()

In [ ]:
optimizer = optax.adabelief(learning_rate=learning_rate)
optimizer_state = optimizer.init(model_moving_params)

In [ ]:
step_count=0
minibatch_idx = 0

In [ ]:
import wandb

In [ ]:
wandb.finish()

In [ ]:
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="04_benchmark_perceptron",    
#     name=model_size_str,
#     # track hyperparameters and run metadata
#     config={
#         'model_param_count':model_param_count,
#         "examples_per_epoch":examples_per_epoch,
#         "batches_per_epoch":batches_per_epoch,
#         "layer_count": 3,
#     }
#     )

In [ ]:
batches_per_epoch

32

In [ ]:
examples_per_epoch

1026734

In [ ]:
train_minibatch_size

32768

In [ ]:
loss_fn = lambda model_params, X,y : jnp.mean(optax.softmax_cross_entropy_with_integer_labels(simpleNN_minibatched.apply(model_params,X),  y))
jitted_val_and_grad = jax.jit(jax.value_and_grad(loss_fn, argnums=0))
examples_seen = 0
for idx_epoch in tqdm(range(1), desc="epoch"):
    for idx_minibatch in tqdm(range(batches_per_epoch), leave=False,  desc="batch in epoch" ):
        step_count+=1
        minibatch_idx = step_count % batches_per_epoch        
        minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)        
        loss_value, gradients = jitted_val_and_grad(model_moving_params, minibatch_X, minibatch_y)
        examples_seen += train_minibatch_size
        model_param_updates, optimizer_state = optimizer.update(gradients, optimizer_state, params=model_moving_params)
        model_moving_params = optax.apply_updates(model_moving_params, model_param_updates)
        # wandb.log({
        #     "step_count":step_count, 
        #     "dataset_seen_ratio":examples_seen/examples_per_epoch, 
        #     "train_loss_value": loss_value,
        #     })   
# wandb.finish()

epoch:   0%|          | 0/1 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
loss_value

Array(2.8855262, dtype=float32)

In [ ]:
starter_prompt = jnp.array(X_train[0])

def predict_text(model_params, prompt_encoded, new_characters=250):
    prediction_decoded = text_decoder(prompt_encoded)
    print(f'{prediction_decoded}')
    print(f'-------------------')
    running_prompt = prompt_encoded.copy()
    continuation = ""
    # print(running_prompt)
    for char_idx in range(new_characters):
        predicted_logits = simpleNN_minibatched.apply(model_params, running_prompt)
        predicted_token = jnp.argmax(predicted_logits).reshape((1,1))
        predicted_char = text_decoder(predicted_token)
        continuation = f'{continuation}{predicted_char}'
        running_prompt = jnp.hstack([running_prompt[1:],predicted_token[:,0]])
        # print(running_prompt)
    print(continuation)    
predict_text(model_moving_params, starter_prompt)


first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to 
-------------------
the tou the the the the the the the that an the the the the the the the the the the the the the the the hat the he the hans the he hans the the he hor the hans the he hans the he hat the he the the the the the he the the the han the the the the the t


In [ ]:
jnp.array(X_test)

(55764, 128)

In [ ]:
# attempt validation accuracy
def validate(model_params, validation_size=4096):
    validation_expected_tokens = jnp.array(y_test[0:validation_size])
    validation_predicted_logits = simpleNN_minibatched.apply(model_params, jnp.array(X_test[0:validation_size]))
    validation_loss = jnp.mean(optax.softmax_cross_entropy_with_integer_labels(validation_predicted_logits, validation_expected_tokens))
    accurate_tokens = jnp.sum(predicted_tokens==validation_expected_tokens)
    accuracy_ratio = accurate_tokens/validation_size
    print(f'validation loss: {validation_loss:0.3f}; accuracy ratio = {accuracy_ratio:0.3f}')

In [ ]:

validation_loss

Array(2.3998523, dtype=float32)

In [ ]:

predicted_tokens.shape

(4096,)

Array(1223, dtype=int32)

In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:
!git add .
!git commit -m 'savegame. now appears to train.'
